In [1]:
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets(
    'MNIST_data', one_hot=True, reshape=False)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.images.shape

(55000, 28, 28, 1)

In [6]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    conv1_weight = tf.get_variable(
        'c1_w', [5, 5, 1, 8], dtype=tf.float32)
    conv1_bias = tf.get_variable(
        'c1_b', [8], dtype=tf.float32)
    conv1_ = tf.nn.conv2d(
        inputs, conv1_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 24, 24, 8]
    conv1 = tf.nn.relu(conv1_ + conv1_bias)
    
    # shape=[None, 12, 12, 8]
    pool1 = tf.nn.max_pool(
        conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    conv2_weight = tf.get_variable(
        'c2_w', shape=[5, 5, 8, 16], dtype=tf.float32)
    conv2_bias = tf.get_variable(
        'c2_b', shape=[16], dtype=tf.float32)
    conv2_ = tf.nn.conv2d(
        pool1, conv2_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 8, 8, 16]
    conv2 = tf.nn.relu(conv2_ + conv2_bias)
    
    # shape=[None, 4, 4, 16]
    pool2 = tf.nn.max_pool(
        conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    tmp = tf.reshape(pool2, [-1, 4 * 4 * 16])
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    
    # 代价函数
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
    
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.5249, acc 0.1389
step   500, loss 0.5152, acc 0.8695
step  1000, loss 0.2519, acc 0.9092
step  1500, loss 0.2390, acc 0.9245
step  2000, loss 0.3164, acc 0.9447
step  2500, loss 0.3313, acc 0.9421
step  3000, loss 0.2031, acc 0.9571
step  3500, loss 0.0677, acc 0.9543
step  4000, loss 0.1619, acc 0.9618
step  4500, loss 0.0909, acc 0.9640
step  5000, loss 0.0346, acc 0.9658
step  5500, loss 0.0627, acc 0.9691
step  6000, loss 0.1395, acc 0.9644
step  6500, loss 0.0250, acc 0.9726
step  7000, loss 0.1206, acc 0.9708
step  7500, loss 0.0171, acc 0.9704
step  8000, loss 0.1180, acc 0.9727
step  8500, loss 0.0081, acc 0.9702
step  9000, loss 0.0846, acc 0.9724
step  9500, loss 0.0399, acc 0.9757
step 10000, loss 0.0602, acc 0.9766
step 10500, loss 0.1542, acc 0.9757
step 11000, loss 0.0368, acc 0.9759
step 11500, loss 0.0146, acc 0.9779
step 12000, loss 0.0905, acc 0.9777
step 12500, loss 0.1214, acc 0.9745
step 13000, loss 0.0873, acc 0.9760
step 13500, loss 0.0085, acc